In [1]:
import schedule
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
baseurl = 'https://www.smartshanghai.com/housing/apartments-rent'

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Function to extract listing IDs
def get_listing_ids():
    # Your existing function to get listing IDs
    pass

# Function to convert time indicator to a timedelta
def convert_time_indicator(time_str):
    if 'hour' in time_str:
        hours = int(time_str.split()[0])
        return timedelta(hours=hours)
    return timedelta()

current_time = datetime.now()
baseurl = "http://example.com"  # Update with the base URL of the listings
listing_ids = get_listing_ids()  # Call your function to get listing IDs

for list_id in listing_ids:
    url = f"{baseurl}/{list_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract time information for the listing
        time_element = soup.find('div', class_='address').text.strip()
        time_str = time_element.split('by')[0].strip()
        
        # Convert time indicator to timedelta
        time_diff = convert_time_indicator(time_str)
        
        # Check if the listing was posted within the last 24 hours
        if time_diff <= timedelta(hours=24):
            # Continue with your scraping and processing logic here
            pass
        else:
            print(f"Listing ID: {list_id} is older than 24 hours, skipping...")
            
    else:
        print(f"Failed to retrieve data for listing ID: {list_id}")

# Once you have collected the relevant listings, proceed with your reporting or data processing logic
# You can store the data in DataFrames and generate reports as needed

In [57]:
def get_listing_id(n):
    Listing_id = []
    Price = []
    N_Bedrooms = []
    N_Bathrooms = []
    for page in range(n):        
        params = {'page': page}
        response = requests.get(baseurl,params)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            content = soup.find_all("div", class_ = 'cont')  
        else:
            print(response.status_code)
        
        for i in range(len(content)):
            Listing_id.append(content[i].find('div').attrs['data-listingid'])
            Price.append(''.join(content[i].find('div',class_="price").text.strip().split()[1].split(',')))
            info = re.findall('\d+',(content[i].find('div', class_ = 'room-type').text.strip()))
            N_Bedrooms.append(info[1])
            N_Bathrooms.append(info[2])
    return pd.DataFrame({'listing_id': Listing_id, 'price': Price,'N_Bedrooms': N_Bedrooms,'N_Bathrooms': N_Bathrooms})

In [121]:
params = {'page': 1}
response = requests.get(baseurl,params)
soup = BeautifulSoup(response.content, "html.parser")
content = soup.find_all("div", class_ = 'cont')

In [122]:
content

[<div class="cont">
 <div class="favorite roundFavoriteButton open-sign-in" data-listingid="1606309"></div>
 <div class="img list-components">
 <ul class="circular-skin housing-list-slider" style="min-height: 166px">
 <li>
 <picture>
 <source data-srcset="https://images.smartshanghai.com.cn/uploads/compressed/2024/08/14/0bef690e-439a-4733-b508-23330d259787.jpg.360.240.webp 360w, https://images.smartshanghai.com.cn/uploads/compressed/2024/08/14/0bef690e-439a-4733-b508-23330d259787.jpg.720.480.webp 720w" sizes="300px" type="image/webp"/>
 <img class="lazy-load" data-src="https://images.smartshanghai.com.cn/uploads/compressed/2024/08/14/0bef690e-439a-4733-b508-23330d259787.jpg.300.200.jpg" height="200" width="300"/>
 </picture>
 </li>
 <li>
 <picture>
 <source data-srcset="https://images.smartshanghai.com.cn/uploads/compressed/2024/08/14/d05e8231-de6a-4997-919d-783d18683918.jpg.360.240.webp 360w, https://images.smartshanghai.com.cn/uploads/compressed/2024/08/14/d05e8231-de6a-4997-919d-783

In [58]:
new = get_listing_id(1)

In [59]:
new

,listing_id,price,N_Bedrooms,N_Bathrooms
0,1606296,12500,2,1
1,1606295,21500,3,2
2,1606293,33500,3,2
3,1606292,19800,3,2
4,1606291,26800,3,2
5,1606288,15000,2,2
6,1606289,17300,3,2
7,1606287,6500,1,1
8,1606285,10000,1,1
9,1606282,5500,1,1


In [160]:
len(new)

15

In [102]:
example = new[:1]

In [114]:
example

,listing_id,price,N_Bedrooms,N_Bathrooms
0,1606296,12500,2,1


In [115]:


columns = []
values = []
amenities_data = []

for list_id in example['listing_id']:
    response = requests.get(f'{baseurl}/{list_id}')
    
    if response.status_code == 200:
        soup_info = BeautifulSoup(response.content, "html.parser")
        
        # Extract the labels and values for the current listing ID
        labels = [label.get_text(strip=True)[:-1] for label in soup_info.find_all('div', class_='details')[0].find_all(name='label')]
        row_values = [value.get_text(strip=True) for value in soup_info.find_all('div', class_='details')[0].find_all(name='div')[:-1]]
        
        # Append unique columns to the columns list
        columns.extend(label for label in labels if label not in columns)
        
        # Append values for the current listing ID while maintaining the correct order
        row = {}
        for label, value in zip(labels, row_values):
            row[label] = value
        values.append(row)
        
        # Extract amenities information
        amenities = soup_info.find_all('div', class_='amenities')[0].find_all('li', class_=['positive', 'negative'])
        amenities_dict = {}
        
        for amenity_li in amenities:
            amenity = amenity_li.text.strip()
            amenities_dict[amenity] = 1 if 'positive' in amenity_li['class'] else 0
        
        amenities_dict['listing_id'] = list_id
        amenities_data.append(amenities_dict)
        
    else:
        print(f"Failed to retrieve data for listing ID: {list_id}")

# Create DataFrames for values and amenities data
values_df = pd.DataFrame(values)
amenities_df = pd.DataFrame(amenities_data)

# Merge the amenities DataFrame with the original DataFrame on 'listing_id'
merged_df = pd.merge(example, amenities_df, on='listing_id', how='left')

# Concatenate the merged DataFrame with the values DataFrame
final_df = pd.concat([merged_df, values_df], axis=1)

In [119]:
final_df.columns = final_df.columns.str.replace("\n", "").str.replace(" ", "")

In [120]:
final_df.columns

Index(['listing_id', 'price', 'N_Bedrooms', 'N_Bathrooms', 'AirFilter',
       'Balcony', 'EnglishSpeakingLandlord', 'FitnessCenters', 'FloorHeating',
       'Garden', 'HistoricBuilding', 'LandlordlivesinShanghai',
       'LargeStorageRoom', 'Oven', 'Parking', 'PetsAllowedfalse', 'Playground',
       'Pool', 'TennisCourts', 'Villa', 'Wallheating', 'WaterFilter', 'Type',
       'AvailableFrom', 'MinimalRentalPeriod', 'DepositRequirement',
       'AdvanceRentPayment', 'AgencyCommission', 'Rooms', 'Size', 'Floor',
       'Furnished', 'MainWindowFacing', 'District', 'Area', 'Compound',
       'MetroStation'],
      dtype='object')

In [94]:
columns = []
values = []

for list_id in example['listing_id']:
    response = requests.get(f'{baseurl}/{list_id}')
    
    if response.status_code == 200:
        soup_info = BeautifulSoup(response.content, "html.parser")
        
        # Extract the labels and values for the current listing ID
        labels = [label.get_text(strip=True)[:-1] for label in soup_info.find_all('div', class_='details')[0].find_all(name='label')]
        row_values = [value.get_text(strip=True) for value in soup_info.find_all('div', class_='details')[0].find_all(name='div')[:-1]]
        
        # Append unique columns to the columns list
        columns.extend(label for label in labels if label not in columns)
        
        # Append values for the current listing ID while maintaining the correct order
        row = {}
        for label, value in zip(labels, row_values):
            row[label] = value
        values.append(row)
        
        # Extract amenities information
        amenities = soup_info.find_all('div', class_='amenities')[0].find_all('li', class_=['positive', 'negative'])
        amenities_dict = {}

        for amenity_li in amenities:
            amenity = amenity_li.text.strip()
            amenities_dict[amenity] = 1 if 'positive' in amenity_li['class'] else 0
            
    else:
        print(f"Failed to retrieve data for listing ID: {list_id}")

new_df = pd.DataFrame(values)
example = pd.concat([example, new_df], axis=1)
amenities_dict['listing_id'] = list_id
amenities_data.append(amenities_dict)

In [101]:
example

,listing_id,price,N_Bedrooms,N_Bathrooms,Type,Available From,Minimal Rental Period,Deposit Requirement,Advance Rent Payment,Agency Commission,Rooms,Size,Floor,Furnished,Main Window Facing,District,Area,Compound,Metro Station
0,1606296,12500,2,1,Apartments,Available Now,2,1,2,35%,"2Bedrooms,1Bathrooms",105 sqm,20,Furnished,South,Jing'an,Jing'an Temple,Jiaozhou Building / 胶州大楼,Jing'an Temple
1,1606295,21500,3,2,Apartments,Available Now,12,2,1,-,"3Bedrooms,2Bathrooms",150 sqm,6,Furnished,South,Jing'an,Jing'an Temple,Sanhe Garden / 三和花园,Jing'an Temple
2,1606293,33500,3,2,Apartments,Available Now,12,2,1,-,"3Bedrooms,2Bathrooms",145 sqm,25,Furnished,South,Jing'an,-,Ladoll International City / 国际丽都城,West Nanjing Rd


In [108]:
for list_id in example['listing_id']:
    response = requests.get(f'{baseurl}/{list_id}')
    soup_info = BeautifulSoup(response.content, "html.parser")
    amenities = soup_info.find_all('div', class_='amenities')[0].find_all('li', class_=['positive', 'negative'])
    
    amenities_dict = {}
    for amenity_li in amenities:
        amenity = amenity_li.text.strip()
        amenities_dict[amenity] = 1 if 'positive' in amenity_li['class'] else 0

In [109]:
amenities_dict

{'Air Filter': 0,
 'Balcony': 1,
 'English Speaking Landlord': 0,
 'Fitness Centers': 0,
 'Floor Heating': 0,
 'Garden': 0,
 'Historic Building': 0,
 'Landlord lives in Shanghai': 1,
 'Large Storage Room': 0,
 'Oven': 1,
 'Parking': 0,
 'Pets Allowed\n\n                                                                                    \n                                                false': 0,
 'Playground': 0,
 'Pool': 0,
 'Tennis Courts': 0,
 'Villa': 0,
 'Wall heating': 0,
 'Water Filter': 0}

In [98]:
# List of columns to clean
columns_to_clean = ['Minimal Rental Period', 'Deposit Requirement', 'Advance Rent Payment']

# Define a function to clean the data in each specified column
def clean_column_data(df, column):
    df[column] = df[column].str.extract('(\d+)').astype(int)

# Apply the cleaning operation to each column in the list
for column in columns_to_clean:
    clean_column_data(example, column)


In [100]:
example['Metro Station'] = example['Metro Station'].str.split('to').str[1].str.split('on').str[0].str.strip()

In [58]:
def details(example):
    for list_id in example['Listing_Id']:
        response = requests.get(f'{baseurl}/{list_id}')
        if response.status_code == 200:
            soup_info = BeautifulSoup(response.content, "html.parser")
            amenities = soup_info.find_all('div', class_='amenities')[0].find_all('li', class_=['positive', 'negative'])

        amenities_dict = {}
        for amenity_li in amenities:
            amenity = amenity_li.text.strip()
            amenities_dict[amenity] = 1 if 'positive' in amenity_li['class'] else 0

                # Add the amenity to the corresponding row in the original DataFrame
        raw.loc[raw['Listing_Id'] == list_id, list(amenities_dict.keys())] = pd.DataFrame([amenities_dict])
        #raw.loc[raw['Listing_Id'] == list_id, amenity_name] = status
    return raw

In [91]:
Amenities = soup_info.find_all('div', class_='amenities')[0].find_all('li', class_=['positive', 'negative'])

In [93]:
amenities_dict = {}
for amenity_li in Amenities:
    amenity = amenity_li.text.strip()
    amenities_dict[amenity] = 1 if 'positive' in amenity_li['class'] else 0
            
            # Add the amenity to the corresponding row in the original DataFrame
raw.loc[raw['Listing_Id'] == list_id, list(amenities_dict.keys())] = pd.DataFrame([amenities_dict])

In [94]:
amenities_dict

{'Air Filter': 0,
 'Balcony': 1,
 'English Speaking Landlord': 0,
 'Fitness Centers': 0,
 'Floor Heating': 1,
 'Garden': 0,
 'Historic Building': 0,
 'Landlord lives in Shanghai': 1,
 'Large Storage Room': 1,
 'Oven': 1,
 'Parking': 0,
 'Pets Allowed\n\n                                                                                    \n                                                true': 1,
 'Playground': 0,
 'Pool': 0,
 'Tennis Courts': 0,
 'Villa': 0,
 'Wall heating': 0,
 'Water Filter': 0}

In [97]:
raw

,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms
1,1605144,11000,60,1,1


In [98]:
pd.DataFrame([amenities_dict])

,Air Filter,Balcony,English Speaking Landlord,Fitness Centers,Floor Heating,Garden,Historic Building,Landlord lives in Shanghai,Large Storage Room,Oven,Parking,Pets Allowed\n\n \n true,Playground,Pool,Tennis Courts,Villa,Wall heating,Water Filter
0,0,1,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,0


In [96]:
raw.merge(pd.DataFrame([amenities_dict]), left_index=True, right_index=True)

,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms,Air Filter,Balcony,English Speaking Landlord,Fitness Centers,Floor Heating,...,Large Storage Room,Oven,Parking,Pets Allowed\n\n \n true,Playground,Pool,Tennis Courts,Villa,Wall heating,Water Filter


In [73]:
raw

,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms,Air Filter,Balcony,English Speaking Landlord,Fitness Centers,Floor Heating,...,Large Storage Room,Oven,Parking,Pets Allowed\n\n \n true,Playground,Pool,Tennis Courts,Villa,Wall heating,Water Filter
1,1605144,11000,60,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
amenities = []
for li in Amenities:
    amenity = li.text.strip()
    status = 1 if 'positive' in li['class'] else 0
    amenities.append({'amenity': amenity, 'status': status})


In [80]:
amenities

[{'amenity': 'Air Filter', 'status': 0},
 {'amenity': 'Balcony', 'status': 1},
 {'amenity': 'English Speaking Landlord', 'status': 0},
 {'amenity': 'Fitness Centers', 'status': 0},
 {'amenity': 'Floor Heating', 'status': 1},
 {'amenity': 'Garden', 'status': 0},
 {'amenity': 'Historic Building', 'status': 0},
 {'amenity': 'Landlord lives in Shanghai', 'status': 1},
 {'amenity': 'Large Storage Room', 'status': 1},
 {'amenity': 'Oven', 'status': 1},
 {'amenity': 'Parking', 'status': 0},
 {'amenity': 'Pets Allowed\n\n                                                                                    \n                                                true',
  'status': 1},
 {'amenity': 'Playground', 'status': 0},
 {'amenity': 'Pool', 'status': 0},
 {'amenity': 'Tennis Courts', 'status': 0},
 {'amenity': 'Villa', 'status': 0},
 {'amenity': 'Wall heating', 'status': 0},
 {'amenity': 'Water Filter', 'status': 0}]

In [48]:
for list_id in raw['Listing_Id']:
    response = requests.get(f'{baseurl}/{list_id}')
    if response.status_code == 200:
        soup_info = BeautifulSoup(response.content, "html.parser")
        amenities = soup_info.find_all('div', class_='amenities')[0].find_all('li', class_=['positive', 'negative'])
        
    amenities_dict = {}
    for amenity_li in amenities:
        amenity = amenity_li.text.strip()
        amenities_dict[amenity] = 1 if 'positive' in amenity_li['class'] else 0
            
    # Add the amenity to the corresponding row in the original DataFrame
    raw.loc[raw['Listing_Id'] == list_id, list(amenities_dict.keys())] = pd.DataFrame([amenities_dict])
    #raw.loc[raw['Listing_Id'] == list_id, amenity_name] = status

In [49]:
raw

,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms,Air Filter,Balcony,English Speaking Landlord,Fitness Centers,Floor Heating,...,Large Storage Room,Oven,Parking,Pets Allowed\n\n \n true,Playground,Pool,Tennis Courts,Villa,Wall heating,Water Filter
0,1605145,10500,85,2,1,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1605144,11000,60,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
